In [1]:
import pandas as pd
import numpy as np
import mysql.connector as conn
from sqlalchemy import create_engine 


In [2]:
# Conexión a la base de datos debes rellenarla contu usuario y contraseña de workbench.
db_conect=conn.connect(host='localhost', user='*******', passwd='*********', database='employees')

In [ ]:
cursor=db_conect.cursor() # cursos para conectarse a la base de datos

In [ ]:
# Prueba para ver si estamos conectados a la base de datos correcta.
cursor.execute('show tables;')

for x in cursor:
    print(x)

# Ejercicio 1.

    Obtener la cantidad de empleados por género de cada departamento.

In [ ]:
gender_employees_per_department = pd.read_sql('''
    select distinct dept_name, 
    count(case employees.gender when 'M' then 1 end) as Male, 
    count(case employees.gender when 'F' then 1 end) as Female
    from employees
    join dept_emp on employees.emp_no = dept_emp.emp_no
    join departments on departments.dept_no = dept_emp.dept_no 

    group by departments.dept_name

    ;
    ''', con=db_conect)
gender_employees_per_department

# Ejercicio 2

    Obtén los nombres de los managers activos y el de su predecesor por cada
    departamento.

In [ ]:
deparment_managers = pd.read_sql('''
Select Deparment, Manager, Previous_Manager from
	(select distinct departments.dept_name as Deparment, 
    concat(employees.first_name, ' ', employees.last_name)as Manager ,
    lag(concat(employees.first_name, ' ', employees.last_name), 1) 
    over (partition by departments.dept_name  order by dept_manager.to_date) as Previous_Manager,
    from_date as Starts_Contract,
    to_date as End_Contract
    from departments

    join dept_manager on departments.dept_no = dept_manager.dept_no
    join employees on dept_manager.emp_no = employees.emp_no) as Managers
where End_Contract = '9999-01-01'
;

    ''', con=db_conect)
deparment_managers

# Ejercicio 3

    Obtén la cantidad de empleados que tiene cada manager.

In [ ]:
employees_per_manager = pd.read_sql('''
    Select Department, 
    concat(employees.first_name, ' ', employees.last_name) as Manager,  
    Number_of_Employees
    from
    (select  departments.dept_name as Department, 
    count(distinct(employees.emp_no))-1 as Number_of_Employees
    from departments
    join dept_manager on departments.dept_no = dept_manager.dept_no
    join dept_emp on departments.dept_no = dept_emp.dept_no
    join employees on dept_emp.emp_no = employees.emp_no
    group by departments.dept_name) as Employees_per_department

    join departments on Employees_per_department.Department = departments.dept_name
    join dept_manager on departments.dept_no = dept_manager.dept_no
    join employees on dept_manager.emp_no = employees.emp_no

    where dept_manager.to_date = '9999-01-01'


    ;

    ''',  con=db_conect)
employees_per_manager

# Ejercicio 4

    Obtén un listado de los empleados con mayor ingreso y con el menor ingreso por
    cada departamento.
    department | employee_max_salary | e_max_salary | employe_min_salary | e_min_salary

In [ ]:
max_and_min_salary_per_department= pd.read_sql('''
    Select Max_Salaries.Department as Department,
    Employee_max_salary,
    Max_salary,
    Employee_min_salary,
    Min_salary
    from
    (select Department, 
    concat(employees.first_name, ' ', employees.last_name) as Employee_max_salary,
    Max_salary
    from
    (Select departments.dept_name as Department, 
    max(salaries.salary) As Max_salary
    from departments
    join dept_emp on departments.dept_no = dept_emp.dept_no
    join employees on dept_emp.emp_no = employees.emp_no
    join salaries on employees.emp_no = salaries.emp_no

    group by departments.dept_name) as Max_Salaries_per_Department

    join departments on Max_Salaries_per_Department.Department = departments.dept_name
    join dept_emp on departments.dept_no = dept_emp.dept_no
    join employees on dept_emp.emp_no = employees.emp_no
    join salaries on employees.emp_no = salaries.emp_no

    where salaries.salary = Max_Salaries_per_Department.Max_salary) as Max_Salaries

    join (select Department, 
    concat(employees.first_name, ' ', employees.last_name) as Employee_min_salary,
    Min_salary
    from
    (Select departments.dept_name as Department, 
    min(salaries.salary) as Min_salary
    from departments
    join dept_emp on departments.dept_no = dept_emp.dept_no
    join employees on dept_emp.emp_no = employees.emp_no
    join salaries on employees.emp_no = salaries.emp_no

    group by departments.dept_name) as Salaries_per_Department

    join departments on Salaries_per_Department.Department = departments.dept_name
    join dept_emp on departments.dept_no = dept_emp.dept_no
    join employees on dept_emp.emp_no = employees.emp_no
    join salaries on employees.emp_no = salaries.emp_no

    where salaries.salary = Salaries_per_Department.Min_salary  ) as Min_Salaries 
    on Max_Salaries.Department = Min_Salaries.Department 
    ;
''', con= db_conect)
max_and_min_salary_per_department